# Scope of Variables

if there are 2 functions with a variable 'var1' and there is another usage of 'var1' in the global scope with the same name there are rules for what variable value 
you are referring to according to the scope. 

In [1]:
function f1()
    x = 11 # local
    println(x)
end

f1 (generic function with 1 method)

In [2]:
f1()

11


In [3]:
x # global

LoadError: UndefVarError: x not defined

In [6]:
x = 22
f1()
x #x global unaffected

11


22

In [10]:
function f2()
    a = 1
    for i in 1:4
        b = i #local to the loop unusable outside
        a = b+1 #refers to function scope, can be used outside and updates that value
    end
    println("a = $a")
    println("b = $b")
end

f2 (generic function with 1 method)

In [11]:
f2()

a = 5


LoadError: UndefVarError: b not defined

# functions within functions

In [14]:
function f3()
    scale = 10
    function f3_inner(x)
        x_inner = x * scale # accesses scale from outer
        println("f3 inner x = $x")
        println("f3 inner x_inner = $x_inner")
    end
    
    f3_inner(10)
    println("from f3 x_inner = $x_inner")
end

f3 (generic function with 1 method)

In [15]:
f3()

f3 inner x = 10
f3 inner x_inner = 100


LoadError: UndefVarError: x_inner not defined

In [16]:
function f()
   i = 0
   for i = 1:3
       # empty
   end
   return i
end;
f()

0

In [25]:
g1 = 10
function f_g1(x)
    global g1 = x #changes g1 outside of function
    g1 *= 10
end

println("g1 = $g1")
f_g1(5)
println("g1 = $g1")

g1 = 10
g1 = 50


In [26]:
g2 = 10
function f_g2(x)
    local g2 = x #changes g1 outside of function
    g2 *= 10
end

println("g2 = $g2")
f_g2(5)
println("g2 = $g2")

g2 = 10
g2 = 10


# Types

The default is that variables are of any type and inferred at run time. Types though can be added afterwards as a refactoring step. 

- In julia all values are true objects with a type from a graph of first class types. 
- :: is put after a variable as a 'type annotation' and can be used to assert a type
- Abstract types are like Number, there is no 'Number' value but can provide a hierarchy for Floats and Integers etc which are below it
- <: means a subtype


In [17]:
1::Int

1

In [18]:
1::String

LoadError: TypeError: in typeassert, expected String, got a value of type Int64

In [19]:
string(1," number one")::String

"1 number one"

In [20]:
Int<:Number

true

In [21]:
1<:Number

LoadError: TypeError: in <:, expected Type, got a value of type Int64

In [22]:
typeof(1)<:Number

true

In [24]:
function f8(x)
    if( typeof(x) <: Number)
        println(x * 10)        
    elseif( typeof(x) <: AbstractString )
        println( "x = $(x*" hi")")
    end
end

f8("say")
f8(10)

x = say hi
100


In [29]:
function f9(x)::Int
    x += 1.5
    return x
end

f9 (generic function with 1 method)

In [30]:
f9(1)

LoadError: InexactError: Int64(2.5)

In [31]:
function f10(x)::Float64
    x += 1.5
    return x
end

f10 (generic function with 1 method)

In [32]:
f10(1)

2.5

In [39]:
function f11(x)::Number
    x += 1.5
    return x
end

f11 (generic function with 1 method)

In [41]:
println( f11(1) )
println( f11(2.5) )

2.5
4.0


## Union types declared from multiple types

In [42]:
Float32or64 = Union{Float32,Float64}

Union{Float32, Float64}

In [45]:
n1::Float16 = 1.5

1.5

In [46]:
typeof(n1)

Float16

In [47]:
n1::Float32or64

LoadError: TypeError: in typeassert, expected Union{Float32, Float64}, got a value of type Float16

In [49]:
n1::Float16

Float16(1.5)

In [50]:
typeof(n1) <: Number

true

In [53]:
struct S1
    x
    y
end

struct S2
    x
    z
end

struct S3
    x
    y
    z
end

structS1orS2 = Union{S1,S2}

s_1 = S1(1,2)
s_2 = S2(1,3)
s_3 = S3(1,2,3)

S3(1, 2, 3)

In [54]:
s_1::S1

S1(1, 2)

In [55]:
s_1::structS1orS2

S1(1, 2)

In [56]:
s_1::S2

LoadError: TypeError: in typeassert, expected S2, got a value of type S1

In [57]:
s_3::structS1orS2

LoadError: TypeError: in typeassert, expected Union{S1, S2}, got a value of type S3

In [58]:
s_3::S3

S3(1, 2, 3)

In [59]:
s_3 isa S3

true

In [60]:
s_3 isa structS1orS2

false

In [61]:
s_1 isa structS1orS2

true

In [62]:
s_3.z

3

In [64]:
#define a generic
struct S4{T}
    x::T
    y::T
end

In [65]:
s4_1 = S4(1,2)

S4{Int64}(1, 2)

In [66]:
s4_2 = S4(1.5,5.5)

S4{Float64}(1.5, 5.5)

In [67]:
s4_3 = S4("hi","friend")

S4{String}("hi", "friend")

In [68]:
#no mixed types on T
s4_4 = S4("hi",30)

LoadError: MethodError: no method matching S4(::String, ::Int64)
[0mClosest candidates are:
[0m  S4(::T, [91m::T[39m) where T at In[64]:3

In [70]:
s4_1::Int64

LoadError: TypeError: in typeassert, expected Int64, got a value of type S4{Int64}

In [71]:
typeof(s4_3).parameters[1]

String

In [72]:
s4_3::S4{String}

S4{String}("hi", "friend")

In [74]:
s4_3::S4{Float64}

LoadError: TypeError: in typeassert, expected S4{Float64}, got a value of type S4{String}

In [75]:
s4_3 isa S4{Float64}

false

In [76]:
s4_3 isa S4{String}

true

In [80]:
function S4_test(s::S4{<:Number})
    println("S4 number type")
end

function S4_test(s::S4{String})
    println("S4 string type")
end

S4_test(s4_3)
S4_test(s4_2)

S4 string type
S4 number type


In [81]:
typeof(1.5)

Float64

In [82]:
s4_2

S4{Float64}(1.5, 5.5)

In [83]:
#make it Float32
s4_4 = S4{Float32}(1.5,5.5)

S4{Float32}(1.5f0, 5.5f0)

In [84]:
typeof( s4_4 )

S4{Float32}

In [85]:
isa(typeof(1.5), Float64)

false

In [86]:
supertype(Float16)

AbstractFloat

In [87]:
supertype(AbstractFloat)

Real

In [88]:
supertype(Real)

Number

In [89]:
supertype(Number)

Any

## optional keyword arguments

In [1]:
f(a=1,b=2) = a+2b

f (generic function with 3 methods)

In [2]:
f(3,3)

9

In [3]:
# same as f(3,2)
f(3)

7

In [4]:
# same as f(1,2)
f()

5

# type parameters

In [8]:
function f12(a::T,b::String,c::T) where {T} 
    return string(a+c," ",b)
end

f12 (generic function with 2 methods)

In [10]:
f12(1,"ok",2)

"3 ok"

In [11]:
#problem if a and c are not the same type
f12(1.0," great",2)

LoadError: MethodError: no method matching f12(::Float64, ::String, ::Int64)
[0mClosest candidates are:
[0m  f12(::T, ::String, [91m::T[39m) where T<:Number at In[5]:1
[0m  f12(::T, ::String, [91m::T[39m) where T at In[8]:1

In [12]:
f12(1.0," great",2.2)

"3.2  great"

### generics and multiple dispatch

In [10]:
function my_multiply(a::T, b::T) where T
    print("foo: ")
    return a * b 
end
function my_multiply(a::T1, b::T2) where {T1,T2}
    print("bar: ")
    return a * b
end
function my_multiply(a::T1, b::T2) where {T1<:Number,T2<:String}
    print("baz: ")
    return string(a) * b 
end
function my_multiply(a::T1, b::T2) where {T1<:String,T2<:Number}
    print("qux: ")
    return a * string(b) 
end

println(my_multiply(2,3)) # foo
println(my_multiply(2.0,3.0)) # foo
println(my_multiply(2,3.0)) # bar
println(my_multiply("2","3")) # foo
println(my_multiply("2",3)) # qux
println(my_multiply(2,"3")) # baz


foo: 6
foo: 6.0
bar: 6.0
foo: 23
qux: 23
baz: 23
